# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37031 instead
  warnings.warn(


Client Scheduler: inproc://10.1.0.145/6693/1 Dashboard: http://10.1.0.145:37031/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,986,Zelda,0.6776958999205376,0.5534070166151499
2000-01-01 00:00:01,1001,Frank,0.42972355096063874,0.6819879096963182
2000-01-01 00:00:02,1095,Ray,-0.8870490837044307,-0.006675175318785476
2000-01-01 00:00:03,965,Michael,0.029346171471789306,0.6510186314559805
2000-01-01 00:00:04,1052,Bob,-0.19915177711755572,-0.6808125942945547
2000-01-01 00:00:05,1052,Ursula,0.903805055389099,-0.6718459509094372
2000-01-01 00:00:06,958,Ursula,0.4578869297128165,-0.7016927026922131
2000-01-01 00:00:07,980,Ray,0.4616883139562673,-0.35445219338733325
2000-01-01 00:00:08,1023,Edith,0.709615477137034,0.581312563036261


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,973,Quinn,0.8970484942796406,0.7308958436715629
2000-01-30 00:00:01,970,Alice,-0.9632225520508988,-0.7048626134855303
2000-01-30 00:00:02,993,Michael,0.3761577474249853,-0.09478349584089463
2000-01-30 00:00:03,1063,Norbert,-0.0835835371670115,-0.41201083719521514
2000-01-30 00:00:04,980,Frank,-0.9490467028763894,0.40267100111039666
2000-01-30 00:00:05,1005,Wendy,0.8689685656901209,0.8010652280932398
2000-01-30 00:00:06,997,Jerry,-0.2788255278659344,0.6808734154152256
2000-01-30 00:00:07,1021,Michael,0.6675853525074895,0.6401897412431037
2000-01-30 00:00:08,1022,George,0.29380148877445267,-0.2470959434119291


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,986,Zelda,0.677696,0.553407
1,2000-01-01 00:00:01,1001,Frank,0.429724,0.681988
2,2000-01-01 00:00:02,1095,Ray,-0.887049,-0.006675
3,2000-01-01 00:00:03,965,Michael,0.029346,0.651019
4,2000-01-01 00:00:04,1052,Bob,-0.199152,-0.680813


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,986,Zelda,0.677696,0.553407
1,2000-01-01 00:00:01,1001,Frank,0.429724,0.681988
2,2000-01-01 00:00:02,1095,Ray,-0.887049,-0.006675
3,2000-01-01 00:00:03,965,Michael,0.029346,0.651019
4,2000-01-01 00:00:04,1052,Bob,-0.199152,-0.680813


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.6 s, sys: 385 ms, total: 3.98 s
Wall time: 4.66 s


name
Alice      -0.000210
Bob         0.000505
Charlie    -0.000979
Dan        -0.000597
Edith       0.000579
Frank       0.001351
George     -0.003603
Hannah     -0.000743
Ingrid     -0.001777
Jerry      -0.001242
Kevin      -0.000207
Laura      -0.000722
Michael     0.000726
Norbert     0.001616
Oliver     -0.002389
Patricia    0.002002
Quinn       0.002622
Ray         0.001092
Sarah      -0.000845
Tim        -0.001304
Ursula      0.000038
Victor      0.000993
Wendy      -0.002207
Xavier      0.001039
Yvonne      0.000870
Zelda      -0.001041
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/pyarrow/compat.py:24: FutureWarning: pyarrow.compat has been deprecated and will be removed in a future release
  warnings.warn("pyarrow.compat has been deprecated and will be removed in a "


In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.11 s, sys: 129 ms, total: 1.24 s
Wall time: 1.31 s


name
Alice      -0.000210
Bob         0.000505
Charlie    -0.000979
Dan        -0.000597
Edith       0.000579
Frank       0.001351
George     -0.003603
Hannah     -0.000743
Ingrid     -0.001777
Jerry      -0.001242
Kevin      -0.000207
Laura      -0.000722
Michael     0.000726
Norbert     0.001616
Oliver     -0.002389
Patricia    0.002002
Quinn       0.002622
Ray         0.001092
Sarah      -0.000845
Tim        -0.001304
Ursula      0.000038
Victor      0.000993
Wendy      -0.002207
Xavier      0.001039
Yvonne      0.000870
Zelda      -0.001041
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 966 ms, sys: 69.9 ms, total: 1.04 s
Wall time: 1.19 s


name
Alice      -0.000210
Bob         0.000505
Charlie    -0.000979
Dan        -0.000597
Edith       0.000579
Frank       0.001351
George     -0.003603
Hannah     -0.000743
Ingrid     -0.001777
Jerry      -0.001242
Kevin      -0.000207
Laura      -0.000722
Michael     0.000726
Norbert     0.001616
Oliver     -0.002389
Patricia    0.002002
Quinn       0.002622
Ray         0.001092
Sarah      -0.000845
Tim        -0.001304
Ursula      0.000038
Victor      0.000993
Wendy      -0.002207
Xavier      0.001039
Yvonne      0.000870
Zelda      -0.001041
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html